In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
import pandas_profiling as pp

# models
from sklearn.linear_model import LinearRegression, SGDRegressor, RidgeCV
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor 
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, VotingRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
import sklearn.model_selection
from sklearn.model_selection import cross_val_predict as cvp
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import lightgbm as lgb

# model tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

import warnings
warnings.filterwarnings("ignore")


In [2]:
valid_part = 0.3
pd.set_option('max_columns',100)

In [3]:
train0 = pd.read_csv('vehicles.csv')
train0.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7119256118,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,https://mohave.craigslist.org,3495,2012.0,jeep,patriot,like new,4 cylinders,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,silver,https://images.craigslist.org/00B0B_k2AXIJ21ok...,"THIS 2012 JEEP PATRIOT IS A 4CYL. AC, STEREO, ...",NaN,az,34.4554,-114.2690
1,7120880186,https://oregoncoast.craigslist.org/cto/d/warre...,oregon coast,https://oregoncoast.craigslist.org,13750,2014.0,bmw,328i m-sport,good,NaN,gas,76237.0,clean,automatic,NaN,rwd,NaN,sedan,grey,https://images.craigslist.org/00U0U_3cLk0WGOJ8...,Selling my 2014 BMW 328i with the following be...,NaN,or,46.1837,-123.8240
2,7115048251,https://greenville.craigslist.org/cto/d/sparta...,greenville / upstate,https://greenville.craigslist.org,2300,2001.0,dodge,caravan,excellent,6 cylinders,gas,199000.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00k0k_t4WqYn5nDC...,"01 DODGE CARAVAN,3.3 ENGINE,AUT TRANS,199000 M...",NaN,sc,34.9352,-81.9654
3,7119250502,https://mohave.craigslist.org/cto/d/lake-havas...,mohave county,https://mohave.craigslist.org,9000,2004.0,chevrolet,colorado ls,excellent,5 cylinders,gas,54000.0,clean,automatic,1GCCS196448191644,rwd,mid-size,pickup,red,https://images.craigslist.org/00J0J_lJEzfeVLHI...,"2004 Chevy Colorado LS, ONLY 54000 ORIGINAL MI...",NaN,az,34.4783,-114.2710
4,7120433904,https://maine.craigslist.org/ctd/d/searsport-t...,maine,https://maine.craigslist.org,0,2021.0,NaN,Honda-Nissan-Kia-Ford-Hyundai-VW,NaN,NaN,other,NaN,clean,other,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/01010_j0IW34mCsm...,CALL: 207.548.6500 TEXT: 207.407.5598 **WE FI...,NaN,me,44.4699,-68.8963


In [4]:
drop_columns = ['id','url', 'region', 'region_url', 'model', 'title_status', 'vin', 'size', 'image_url', 'description', 'lat','long','county']
train0 = train0.drop(columns = drop_columns)

In [5]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435849 entries, 0 to 435848
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         435849 non-null  int64  
 1   year          434732 non-null  float64
 2   manufacturer  415102 non-null  object 
 3   condition     249043 non-null  object 
 4   cylinders     269465 non-null  object 
 5   fuel          432858 non-null  object 
 6   odometer      360701 non-null  float64
 7   transmission  433703 non-null  object 
 8   drive         313838 non-null  object 
 9   type          318741 non-null  object 
 10  paint_color   300602 non-null  object 
 11  state         435849 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 39.9+ MB


In [6]:
train0 = train0.dropna()
train0.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
3,9000,2004.0,chevrolet,excellent,5 cylinders,gas,54000.0,automatic,rwd,pickup,red,az
7,8500,2005.0,ford,excellent,6 cylinders,gas,62800.0,automatic,rwd,convertible,silver,me
12,2750,2006.0,chevrolet,good,8 cylinders,gas,165000.0,automatic,4wd,truck,white,me
19,24930,2017.0,subaru,excellent,4 cylinders,gas,32989.0,automatic,4wd,SUV,grey,mt
26,3200,1998.0,volkswagen,good,4 cylinders,gas,98186.0,manual,rwd,coupe,blue,mt


In [7]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = train0.columns.values.tolist()
for col in features:
    if train0[col].dtype in numerics: continue
    categorical_columns.append(col)
# Encoding categorical features
for col in categorical_columns:
    if col in train0.columns:
        le = LabelEncoder()
        le.fit(list(train0[col].astype(str).values))
        train0[col] = le.transform(list(train0[col].astype(str).values))



In [8]:
train0['year'] = (train0['year']-1900).astype(int)
train0['odometer'] = train0['odometer'].astype(int)

In [9]:
train0.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
3,9000,104,7,0,4,2,54000,0,2,8,8,3
7,8500,105,13,0,5,2,62800,0,2,2,9,21
12,2750,106,7,2,6,2,165000,0,0,10,10,21
19,24930,117,37,0,3,2,32989,0,0,0,5,26
26,3200,98,40,2,3,2,98186,1,2,3,1,26


In [10]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124793 entries, 3 to 435846
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   price         124793 non-null  int64
 1   year          124793 non-null  int32
 2   manufacturer  124793 non-null  int64
 3   condition     124793 non-null  int64
 4   cylinders     124793 non-null  int64
 5   fuel          124793 non-null  int64
 6   odometer      124793 non-null  int32
 7   transmission  124793 non-null  int64
 8   drive         124793 non-null  int64
 9   type          124793 non-null  int64
 10  paint_color   124793 non-null  int64
 11  state         124793 non-null  int64
dtypes: int32(2), int64(10)
memory usage: 11.4 MB


In [11]:
train0['price'].value_counts()

0        4552
4500     1403
5995     1358
3500     1322
6995     1316
         ... 
5472        1
19755       1
7457        1
7393        1
19734       1
Name: price, Length: 5547, dtype: int64

In [12]:
train0 = train0[train0['price'] > 1000]
train0 = train0[train0['price'] < 40000]
# Rounded ['odometer'] to 5000
train0['odometer'] = train0['odometer'] // 5000
train0 = train0[train0['year'] > 110]

In [13]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60870 entries, 19 to 435840
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   price         60870 non-null  int64
 1   year          60870 non-null  int32
 2   manufacturer  60870 non-null  int64
 3   condition     60870 non-null  int64
 4   cylinders     60870 non-null  int64
 5   fuel          60870 non-null  int64
 6   odometer      60870 non-null  int32
 7   transmission  60870 non-null  int64
 8   drive         60870 non-null  int64
 9   type          60870 non-null  int64
 10  paint_color   60870 non-null  int64
 11  state         60870 non-null  int64
dtypes: int32(2), int64(10)
memory usage: 5.6 MB


In [14]:
train0.corr()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
price,1.000000,0.451994,-0.028563,0.115130,0.500969,-0.181776,-0.242929,0.182608,-0.245597,0.034775,0.042705,0.004828
year,0.451994,1.000000,0.039877,0.133841,-0.096874,0.065522,-0.362668,0.093475,-0.020978,0.012367,0.042515,-0.014672
manufacturer,-0.028563,0.039877,1.000000,-0.008770,-0.200279,-0.056015,-0.030149,0.042601,-0.099195,0.050371,-0.018589,-0.006306
condition,0.115130,0.133841,-0.008770,1.000000,0.053672,0.030514,-0.071651,0.143540,0.033947,0.027081,0.002252,-0.000423
cylinders,0.500969,-0.096874,-0.200279,0.053672,1.000000,-0.098038,0.071667,0.102699,-0.141777,0.072877,0.042781,0.021634
fuel,-0.181776,0.065522,-0.056015,0.030514,-0.098038,1.000000,-0.104450,0.087760,0.081593,-0.119522,-0.049588,-0.032950
odometer,-0.242929,-0.362668,-0.030149,-0.071651,0.071667,-0.104450,1.000000,-0.114799,-0.031043,0.034483,0.020085,0.014536
transmission,0.182608,0.093475,0.042601,0.143540,0.102699,0.087760,-0.114799,1.000000,0.045928,0.002393,-0.019678,-0.024077
drive,-0.245597,-0.020978,-0.099195,0.033947,-0.141777,0.081593,-0.031043,0.045928,1.000000,0.124981,0.075445,-0.087186
type,0.034775,0.012367,0.050371,0.027081,0.072877,-0.119522,0.034483,0.002393,0.124981,1.000000,0.087766,-0.002102


In [15]:
train0.describe()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
count,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000,60870.000000
mean,15983.185313,114.283177,18.993264,1.152012,4.398899,1.927567,16.134483,0.159619,0.720831,6.113603,5.709479,24.100230
std,8397.447147,2.304149,11.495406,1.234212,1.276746,0.530551,15.605430,0.502718,0.731753,4.154347,4.062670,15.019661
min,1061.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,112.000000,10.000000,0.000000,3.000000,2.000000,8.000000,0.000000,0.000000,0.000000,1.000000,9.000000
50%,14250.000000,114.000000,14.000000,0.000000,5.000000,2.000000,15.000000,0.000000,1.000000,8.000000,8.000000,23.000000
75%,21395.000000,116.000000,31.000000,2.000000,5.000000,2.000000,22.000000,0.000000,1.000000,9.000000,10.000000,37.000000
max,39999.000000,121.000000,41.000000,5.000000,7.000000,4.000000,1629.000000,2.000000,2.000000,12.000000,11.000000,50.000000


In [16]:
#pp.ProfileReport(train0)

In [17]:
target_name = 'price'
train_target0 = train0[target_name]
train0 = train0.drop([target_name], axis=1)

In [18]:
train0, test0, train_target0, test_target0 = train_test_split(train0, train_target0, test_size=0.2, random_state=0)

In [19]:
# For boosting model
train0b = train0
train_target0b = train_target0
# Synthesis valid as test for selection models
trainb, testb, targetb, target_testb = train_test_split(train0b, train_target0b, test_size=valid_part, random_state=0)

In [20]:
scaler = StandardScaler()
train0 = pd.DataFrame(scaler.fit_transform(train0), columns = train0.columns)

In [21]:
train0.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
0,0.311650,-1.040793,0.688127,1.252857,0.137755,0.305670,-0.315523,1.745705,0.934635,1.056601,1.458450
1,-0.557418,1.745613,0.688127,0.470659,0.137755,-0.396938,3.672723,-0.985372,0.453174,-1.405476,-1.540751
2,0.311650,-0.518342,0.688127,0.470659,0.137755,0.227603,1.678600,-0.985372,0.934635,-0.174438,0.058823
3,2.049786,-0.518342,1.498484,-1.093738,0.137755,-1.099547,-0.315523,0.380167,0.693905,1.056601,0.325418
4,-0.557418,-0.431267,-0.932585,1.252857,0.137755,1.086346,-0.315523,1.745705,-1.472670,1.056601,1.258503


In [22]:
train, test, target, target_test = train_test_split(train0, train_target0, test_size=valid_part, random_state=0)

In [23]:
train.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
27855,-0.991952,-0.518342,-0.932585,1.252857,-3.637222,-0.631141,-0.315523,-0.985372,0.934635,-1.405476,-1.007560
27348,-1.426486,1.571463,-0.932585,-1.093738,0.137755,0.071467,-0.315523,-0.985372,-1.472670,-1.405476,0.791961
30708,0.311650,1.745613,0.688127,-1.093738,0.137755,-0.084668,-0.315523,0.380167,-0.509748,0.810393,0.858610
15386,0.746184,0.091184,-0.932585,0.470659,0.137755,0.383738,-0.315523,-0.985372,-1.472670,0.564185,0.192121
37459,0.746184,-0.170041,-0.932585,-1.093738,0.137755,-1.021479,-0.315523,-0.985372,-1.472670,0.810393,0.725312


In [24]:
test.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state
46151,2.049786,-0.518342,-0.932585,0.470659,0.137755,-1.099547,-0.315523,1.745705,1.175366,1.056601,-0.274422
11732,-1.426486,-0.518342,-0.932585,0.470659,0.137755,1.476685,-0.315523,-0.985372,-1.472670,0.317978,0.925259
41922,-0.557418,1.049012,1.498484,0.470659,0.137755,0.696008,-0.315523,0.380167,0.693905,-0.174438,0.458716
18915,-0.991952,-1.040793,-0.932585,0.470659,0.137755,-0.162735,-0.315523,0.380167,0.693905,-1.405476,-0.341071
22745,0.311650,-0.779568,-0.932585,-1.093738,0.137755,-0.084668,-0.315523,0.380167,0.693905,0.810393,0.858610


In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34087 entries, 27855 to 2732
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          34087 non-null  float64
 1   manufacturer  34087 non-null  float64
 2   condition     34087 non-null  float64
 3   cylinders     34087 non-null  float64
 4   fuel          34087 non-null  float64
 5   odometer      34087 non-null  float64
 6   transmission  34087 non-null  float64
 7   drive         34087 non-null  float64
 8   type          34087 non-null  float64
 9   paint_color   34087 non-null  float64
 10  state         34087 non-null  float64
dtypes: float64(11)
memory usage: 3.1 MB


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14609 entries, 46151 to 30951
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          14609 non-null  float64
 1   manufacturer  14609 non-null  float64
 2   condition     14609 non-null  float64
 3   cylinders     14609 non-null  float64
 4   fuel          14609 non-null  float64
 5   odometer      14609 non-null  float64
 6   transmission  14609 non-null  float64
 7   drive         14609 non-null  float64
 8   type          14609 non-null  float64
 9   paint_color   14609 non-null  float64
 10  state         14609 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB
